# Работа со строковыми значениями

__Автор задач: Блохин Н.В. (NVBlokhin@fa.ru)__

Материалы:
* Макрушин С.В. Лекция "Работа со строковыми значениям"
* https://pyformat.info/
* https://docs.python.org/3/library/re.html
    * https://docs.python.org/3/library/re.html#flags
    * https://docs.python.org/3/library/re.html#functions
* https://pythonru.com/primery/primery-primeneniya-regulyarnyh-vyrazheniy-v-python
* https://kanoki.org/2019/11/12/how-to-use-regex-in-pandas/
* https://realpython.com/nltk-nlp-python/

## Задачи для совместного разбора

1. Вывести на экран данные из словаря `obj` построчно в виде `k = v`, задав формат таким образом, чтобы знак равенства оказался на одной и той же позиции во всех строках. Строковые литералы обернуть в кавычки.

In [ ]:
obj = {
    "home_page": "https://github.com/pypa/sampleproject",
    "keywords": "sample setuptools development",
    "license": "MIT",
}

for k, v in obj.items():
    print('{:20} =   \"{}\"'.format(k, v))

home_page            =   "https://github.com/pypa/sampleproject"
keywords             =   "sample setuptools development"
license              =   "MIT"


2. Написать регулярное выражение,которое позволит найти номера групп студентов.

In [ ]:
import pandas as pd
import re 

obj = pd.Series(["Евгения гр.ПМ19-1", "Илья пм 20-4", "Анна 20-3"])

pattern = r'\d{2}-\d'

for string in obj:
    match = re.search(pattern, string)
    if match:
        print(match.group(0))


19-1
20-4
20-3


3. Разбейте текст формулировки задачи 2 на слова.

In [ ]:
text = "Написать регулярное выражение,которое позволит найти номера групп студентов."
words = re.findall(r'\w+', text)
print(words)



['Написать', 'регулярное', 'выражение', 'которое', 'позволит', 'найти', 'номера', 'групп', 'студентов']


4. Найдите в тексте все последовательности: "вол, воз, вон".
 
Текст: "вол воБ во8 воз вок вог во4 воХ во! воь вон" (От Калажоков З.Х.)

In [ ]:
text = "вол воБ во8 воз вок вог во4 воХ во! воь вон"
pattern = r'во[лзн]'

matches = re.findall(pattern, text)
print(matches)


['вол', 'воз', 'вон']


5. Напишите регулярное выражение, которое найдёт все кабинеты с трёхзначным номером: 100 - 999 в строке 

'147 кабинет 843 кабинет 010 кабинет 514 кабинет 99 кабинет 246 кабинет 572 кабинет'. 

Шаблон кабинета: ddd кабинет, где d - арабская цифра. (От Калажоков З.Х.)

In [ ]:
qwerty = '147 кабинет 843 кабинет 010 кабинет 514 кабинет 99 кабинет 246 кабинет 572 кабинет'
q = qwerty.split(' ')

pattern = r'\d{3}'

for string in q:
    m = re.search(pattern, string)
    if m:
        print(m.group(0), 'кабинет')

147 кабинет
843 кабинет
010 кабинет
514 кабинет
246 кабинет
572 кабинет


## Лабораторная работа 6

### Форматирование строк

1\. Загрузите данные из файла `recipes_sample.csv` (__ЛР2__) в виде `pd.DataFrame` `recipes` При помощи форматирования строк выведите информацию об id рецепта и времени выполнения 5 случайных рецептов в виде таблицы следующего вида:

    
    |      id      |  minutes  |
    |--------------------------|
    |    61178     |    65     |
    |    202352    |    80     |
    |    364322    |    150    |
    |    26177     |    20     |
    |    224785    |    35     |
    
Обратите внимание, что ширина столбцов заранее неизвестна и должна рассчитываться динамически, в зависимости от тех данных, которые были выбраны. 

In [ ]:
import pandas as pd

recipes = pd.read_csv('recipes_sample.csv', delimiter=',')
new = recipes.sample(n=5)

q = []
for i in recipes['id']:
    q.append(len(str(i)))
a = int(max(q))

print('|{:^{}}|{:^{}}|'.format('id', a*2, ' minutes', a*2))
print('|{}|'.format('--'* a * 2 + '-'))


for _, row in new.iterrows():
    print('|{:^{}}|{:^{}}|'.format(row['id'], a*2, row['minutes'], a*2))

|     id     |   minutes  |
|-------------------------|
|   32602    |     50     |
|   381137   |     20     |
|   202187   |     75     |
|   107387   |     75     |
|   200298   |     65     |


2\. Напишите функцию `show_info`, которая по данным о рецепте создает строку (в смысле объекта python) с описанием следующего вида:

```
"Название Из Нескольких Слов"

1. Шаг 1
2. Шаг 2
----------
Автор: contributor_id
Среднее время приготовления: minutes минут
```

    
Данные для создания строки получите из файлов `recipes_sample.csv` (__ЛР2__) и `steps_sample.xml` (__ЛР3__). 
Вызовите данную функцию для рецепта с id `170895` и выведите (через `print`) полученную строку на экран.

In [ ]:
from bs4 import BeautifulSoup as bs

with open('steps_sample.xml', 'r') as f: 
    steps_sample = bs(f, 'xml')

def show_info(n):
    res = f'{recipes[recipes.id == n].name.iloc[0].title()} \n \n'
    
    for r in steps_sample.recipes.find_all('recipe'):
        if r.find('id').next == str(n):
            steps = [step.next for step in r.steps.find_all('step')]
    for i, step in enumerate(steps):
        res += f'{i + 1}. {step.capitalize()} \n'
        
    res += '\n' + '--'*10 + '\n' + '\n' + f'Автор:  {recipes[recipes.id == n].contributor_id.iloc[0]}' + '\n' + f'Среднее время приготовления:  {recipes[recipes.id == n].minutes.iloc[0]} минут'
    return print(f'{res}')

In [ ]:
show_info(170895)


Leeks And Parsnips  Sauteed Or Creamed 
 
1. Clean the leeks and discard the dark green portions 
2. Cut the leeks lengthwise then into one-inch pieces 
3. Melt the butter in a medium skillet , med 
4. Heat 
5. Add the garlic and fry 'til fragrant 
6. Add leeks and fry until the leeks are tender , about 6-minutes 
7. Meanwhile , peel and chunk the parsnips into one-inch pieces 
8. Place in a steaming basket and steam 'til they are as tender as you prefer 
9. I like them fork-tender 
10. Drain parsnips and add to the skillet with the leeks 
11. Add salt and pepper 
12. Gently sautee together for 5-minutes 
13. At this point you can serve it , or continue on and cream it: 
14. In a jar with a screw top , add the half-n-half and arrowroot 
15. Shake 'til blended 
16. Turn heat to low under the leeks and parsnips 
17. Pour in the arrowroot mixture , stirring gently as you pour 
18. If too thick , gradually add the water 
19. Let simmer for a couple of minutes 
20. Taste to adjust seasoning

In [ ]:
recipes[recipes.id == 170895]


,name,id,minutes,contributor_id,submitted,n_steps,description,n_ingredients
15754,leeks and parsnips sauteed or creamed,170895,27,8377,2006-05-31,21.0,this is good sauteed only or creamed. very eas...,9.0


## Работа с регулярными выражениями

3\. Напишите регулярное выражение, которое ищет следующий паттерн в строке: число (1 цифра или более), затем пробел, затем слова: hour или hours или minute или minutes. Произведите поиск по данному регулярному выражению в каждом шаге рецепта с id 25082. Выведите на экран все непустые результаты, найденные по данному шаблону.

In [ ]:
import re

for r in steps_sample.recipes.find_all('recipe'):
    if r.find('id').next == '25082':
        steps = [step.next for step in r.steps.find_all('step')]
        
        for i, step in enumerate(steps):
            matches = re.findall(r'\d+\s+(?:hours?|minutes?)', step)
            if matches:
                print(matches)

['20 minutes']
['10 minutes']
['2 hours']
['10 minutes']
['20 minutes', '30 minutes']


4\. Напишите регулярное выражение, которое ищет шаблон вида "this..., but" _в начале строки_ . Между словом "this" и частью ", but" может находиться произвольное число букв, цифр, знаков подчеркивания и пробелов. Никаких других символов вместо многоточия быть не может. Пробел между запятой и словом "but" может присутствовать или отсутствовать.

Используя строковые методы `pd.Series`, выясните, для каких рецептов данный шаблон содержится в тексте описания. Выведите на экран количество таких рецептов и 3 примера подходящих описаний (текст описания должен быть виден на экране полностью).

In [ ]:
recipe = recipes['description'].squeeze(axis=0).dropna()
d=re.compile(r'^this[a-zA-Zа-яЁА-ЯЁ0-9_ ]*, but')
matches=[]
for i in recipe:
  a=d.findall(i)
  if len(a)>0:
   matches.append(i)
   ser=pd.Series(matches)
print(f'Количество подходящих рецептов: {len(ser)}\n')
for i in range(3):
  print(ser[i]+'\n')


Количество подходящих рецептов: 130

this was adapted from a recipe i found on the net, but i added julienne onion to the peppers.  this is a meal in itself, or you could have a small slice with a meat dish.  for those that like to have brunch, it's a bit different to your traditional quiche recipes.  if you love cheese, you could add 1/2 cup of your favorite to the egg mixture, then pour over peppers.

this is kind of similar to some of the other versions out there, but it is the best and easiest i have found

this is a moist, buttery apple cake.  the dough is best made the night before, put in the fridge and then filled and baked the next morning.



5\. В текстах шагов рецептов обыкновенные дроби имеют вид "a / b". Используя регулярные выражения, уберите в тексте шагов рецепта с id 72367 пробелы до и после символа дроби. Выведите на экран шаги этого рецепта после их изменения.

In [ ]:
bakugo = []
step = []

for reciep in steps_sample.recipes.find_all('recipe'):
    steps = [step.next for step in reciep.steps.find_all('step')]
    bakugo.append({reciep.find('id').next: steps})
    if reciep.find('id').next == '72367':
        step = steps
        
result = [re.sub(r'\b(\d+)\s*/\s*(\d+)\b', r'\1/\2', i, flags=re.MULTILINE) for i in step]

result

['mix butter , flour , 1/3 c',
 'sugar and 1-1/4 t',
 'vanilla',
 'press into greased 9" springform pan',
 'mix cream cheese , 1/4 c',
 'sugar , eggs and 1/2 t',
 'vanilla beating until fluffy',
 'pour over dough',
 'combine apples , 1/3 c',
 'sugar and cinnamon',
 'arrange on top of cream cheese mixture and sprinkle with almonds',
 'bake at 350 for 45-55 minutes , or until tester comes out clean']

### Сегментация текста

6\. Разбейте тексты шагов рецептов на слова при помощи пакета `nltk`. Посчитайте и выведите на экран кол-во уникальных слов среди всех рецептов. Словом называется любая последовательность алфавитных символов (для проверки можно воспользоваться `str.isalpha`). При подсчете количества уникальных слов не учитывайте регистр.

In [ ]:
import re
import xml.etree.ElementTree as ET
tk = ToktokTokenizer()

tree = ET.parse('steps_sample.xml')
root = tree.getroot()
a = ''
matches=[]
for recipe in root:
  b= ' '.join([recipe[1][i].text for i in range(len(recipe[1]))]).lower()
  matches+=[i for i in tk.tokenize(b) if i.isalpha()==True]
len(set(matches))

14956

7\. Разбейте описания рецептов из `recipes` на предложения при помощи пакета `nltk`. Найдите 5 самых длинных описаний (по количеству _предложений_) рецептов в датасете и выведите строки фрейма, соответствующие этим рецептами, в порядке убывания длины.

In [ ]:
!pip install nltk
%pip install razdel
from razdel import sentenize
import nltk
from nltk.tokenize import sent_tokenize
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [ ]:
recipes['des_sen_num']=[len(list(sentenize(str(i)))) for i in recipes['description']]
recipes.sort_values('des_sen_num', ascending=False).drop('des_sen_num',axis=1)[:5]

,name,id,minutes,contributor_id,submitted,n_steps,description,n_ingredients
12948,grilled pb j with apples,300031,17,678862,2008-04-22,8.0,from family fun magazine. this is one of my k...,6.0
19238,onion stuffed flat bread,443204,30,1191081,2010-12-01,NaN,from vegetarian times. pan-fried flat breads a...,7.0
29833,zesty edamame,470976,25,599450,2011-12-31,8.0,this recipe is based on one from first magazin...,4.0
23954,sherried artichoke chicken,284885,90,755161,2008-02-07,11.0,i got this from my mother. i remember her mak...,8.0
22566,rich barley mushroom soup,328708,60,221776,2008-10-03,NaN,this is one of the best soups i've ever made a...,10.0


8\. Напишите функцию, которая для заданного предложения выводит информацию о частях речи слов, входящих в предложение, в следующем виде:
```
PRP   VBD   DT      NNS     CC   VBD      NNS        RB   
 I  omitted the raspberries and added strawberries instead
``` 
Для определения части речи слова можно воспользоваться `nltk.pos_tag`.

Проверьте работоспособность функции на названии рецепта с id 241106.

Обратите внимание, что часть речи должна находиться ровно посередине над соотвествующим словом, а между самими словами должен быть ровно один пробел.


In [ ]:
from nltk.tag import pos_tag

def tags(s):
    tag = pos_tag(tkz.tokenize(s))
    paddings = [max(len(i), len(k)) for i, k in tag]

    up = ''
    low = ''

    for p, (i, t) in zip(paddings, tag):
        up += f"{t:^{p+2}}"
        low += f"{i:^{p+2}}"
    print(up)
    print(low)

tags(recipes[recipes.id == 241106].name.values[0])

    JJ      NNS     IN      NNS     VBP     JJ    CC    JJ     NNS   
 eggplant  steaks  with  chickpeas  feta  cheese  and  black  olives 
